# 02 — Feature Preparation
Mục tiêu: đọc `cleaned.parquet`, kiểm lại leakage, chọn cột, và lưu snapshot modelling-ready `data/processed/dataset_for_clf.parquet`.

In [1]:
CLEANED_PATH = 'data/processed/cleaned.parquet'
OUTPUT_DATASET_PATH = 'data/processed/dataset_for_clf.parquet'

# (tuỳ chọn) lọc những dòng thiếu target
DROP_ROWS_WITHOUT_TARGET = True


In [2]:
# Parameters
CLEANED_PATH = "data/processed/cleaned.parquet"
OUTPUT_DATASET_PATH = "data/processed/dataset_for_clf.parquet"
DROP_ROWS_WITHOUT_TARGET = True


In [3]:
from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path('..').resolve()
cleaned_path = (PROJECT_ROOT / CLEANED_PATH).resolve()
out_path = (PROJECT_ROOT / OUTPUT_DATASET_PATH).resolve()
out_path.parent.mkdir(parents=True, exist_ok=True)

df = pd.read_parquet(cleaned_path)
print('loaded:', cleaned_path)
print('shape:', df.shape)
df.head()

loaded: D:\KPDL\lab4\data\processed\cleaned.parquet
shape: (420768, 55)


,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,...,PM10_lag24,SO2_lag24,NO2_lag24,CO_lag24,O3_lag24,TEMP_lag24,PRES_lag24,DEWP_lag24,RAIN_lag24,WSPM_lag24
0,1,2013,3,1,0,4.0,4.0,4.0,7.0,300.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2013,3,1,1,8.0,8.0,4.0,7.0,300.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2013,3,1,2,7.0,7.0,5.0,10.0,300.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2013,3,1,3,6.0,6.0,11.0,11.0,300.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2013,3,1,4,3.0,3.0,12.0,12.0,300.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Check leakage columns (PM2.5 & pm25_24h phải loại khỏi feature khi train)
leak_cols = [c for c in ['PM2.5', 'pm25_24h'] if c in df.columns]
print('Leakage columns present (OK, but will be excluded in modelling):', leak_cols)

if DROP_ROWS_WITHOUT_TARGET:
    before = len(df)
    df = df[df['aqi_class'].notna()].copy()
    print('dropped rows without target:', before - len(df))


Leakage columns present (OK, but will be excluded in modelling): ['PM2.5', 'pm25_24h']


dropped rows without target: 7833


In [5]:
# Gợi ý feature set (để sinh viên thấy rõ)
drop_cols = {'PM2.5', 'pm25_24h', 'aqi_class', 'datetime'}
feature_cols = [c for c in df.columns if c not in drop_cols]
print('n_features:', len(feature_cols))
feature_cols[:30]

n_features: 51


['No',
 'year',
 'month',
 'day',
 'hour',
 'PM10',
 'SO2',
 'NO2',
 'CO',
 'O3',
 'TEMP',
 'PRES',
 'DEWP',
 'RAIN',
 'wd',
 'WSPM',
 'station',
 'hour_sin',
 'hour_cos',
 'dow',
 'is_weekend',
 'PM10_lag1',
 'SO2_lag1',
 'NO2_lag1',
 'CO_lag1',
 'O3_lag1',
 'TEMP_lag1',
 'PRES_lag1',
 'DEWP_lag1',
 'RAIN_lag1']

In [6]:
df.to_parquet(out_path, index=False)
print('Saved:', out_path)

Saved: D:\KPDL\lab4\data\processed\dataset_for_clf.parquet
